In [13]:
import polars as pl
import pyodbc
from datetime import date

import time
start_time = time.time()

In [14]:
from IPython import get_ipython
get_ipython().run_line_magic("run", "functions.ipynb")

#### Definición de Rutas

In [15]:
cierre = "202502"
cierre_l = "Feb25"

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"

# Archivos
fl_desembolsos_p1 = wd_data_raw + f'Desembolsos_P1_{cierre}'
fl_desembolsos_p1_bmxt = wd_data_raw + f'Desembolsos_P1_80686_{cierre}'
fl_desembolsos_p1_fianzas = wd_data_raw + f'Desembolsos_P1_BMXT_{cierre}'
fl_desembolsos_p2 = wd_data_raw + f'Desembolsos_P2_{cierre}'
fl_desembolsos_p2_bmxt = wd_data_raw + f'Desembolsos_P2_80686_{cierre}'
fl_desembolsos_p2_fianzas = wd_data_raw + f'Desembolsos_P2_BMXT_{cierre}'
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_recuperaciones = wd_data_raw + f'Recuperaciones_{cierre}.csv'
fl_recuperaciones_bmxt = wd_data_raw + f'Recuperaciones_BMXT_{cierre}.mdb'


### Catálogos

In [16]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

# Solo Recuperaciones
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)

# Recuperaciones, Pagos
tipo_cambio = pl.read_database(query='SELECT * FROM [TIPO CAMBIO]', connection=conn)
programa = pl.read_database(query='SELECT * FROM [PROGRAMA]', connection=conn, schema_overrides={'PROGRAMA_ID': pl.Int128})
agrupamiento = pl.read_database(query='SELECT * FROM [AGRUPAMIENTO]', connection=conn)
udis = pl.read_database(query='SELECT * FROM [UDIS]', connection=conn, schema_overrides={'Fecha_Paridad': pl.Datetime})
#udis = udis.with_columns(pl.col("Fecha_Paridad").cast(pl.Date))
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

conn.close()

tipo_cambio.columns = ["Anio", "Mes", "TC"]
cierre = date(2024,12,31)
tdc = tipo_cambio.filter((pl.col("Anio")==cierre.year) & (pl.col("Mes")==cierre.month)).select("TC").item()
tdc

20.8829

### Querie Pagos

In [ ]:
def main_querie_pagadas():
    pagos_f1 = import_f1()
    pagos_f2 = import_f2()

    # Crea Concatenados y TPRO_CLAVE
    pagos_f1 = genera_concatenado(pagos_f1)
    pagos_f1 = genera_tpro_clave(pagos_f1)
    pagos_f2 = genera_concatenado(pagos_f2)

    pagadas_global_inter = genera_pagadas_global_inter(pagos_f1, pagos_f2)
    pagadas_global_vf = genera_pagadas_global_vf(pagadas_global_inter)
    pagadas_global_vf = complementa_pagadas_global_vf(pagadas_global_vf)

    valida_pagos = genera_validador(pagos_f2)
    pagadas_global_vf_bancomext = genera_pagadas_global_bancomext(pagadas_global_vf)
    pagadas_global_vf_sin_bancomext = genera_pagadas_global_sin_bancomext(pagadas_global_vf)

    valida_base_pagos = genera_valida_base_pagos(pagadas_global_vf)
    valida_base_pagos_mn = genera_valida_base_pagos_mn(pagadas_global_vf)

    #pagadas_global_vf.write_csv('./../data/processed/DWH/pagadas_global_vf.csv')
    #pagadas_global_vf_bancomext.write_csv('./../data/processed/DWH/pagadas_global_vf_bancomext.csv')
    #pagadas_global_vf_sin_bancomext.write_csv('./../data/processed/DWH/pagadas_global_vf_sin_bancomext.csv')
    #valida_pagos.write_csv('./../data/processed/DWH/valida_pagos.csv')
    #valida_base_pagos.write_csv('./../data/processed/DWH/valida_base_pagos.csv')
    #valida_base_pagos_mn.write_csv('./../data/processed/DWH/valida_base_pagos_mn.csv')

    return pagadas_global_vf

#pagadas_global_vf = main_querie_pagadas()

### Querie Recuperaciones

#### Ejecución

In [ ]:
def main_querie_recuperaciones():
    dwh_recuperaciones = importa_recuperaciones()
    recuperadas_global_inter = genera_recuperadas_global_inter(dwh_recuperaciones)
    recuperadas_global_vf = genera_recuperadas_global_vf(recuperadas_global_inter)

    recuperadas_global_vf_bancomext = genera_recuperadas_global_bancomext(recuperadas_global_vf)
    recuperadas_global_vf_sin_bancomext = genera_recuperadas_global_sin_bancomext(recuperadas_global_vf)

    valida_dwh_dac = genera_valida_dwh_dac(dwh_recuperaciones)
    valida_td = genera_valida_td(dwh_recuperaciones)

    #recuperadas_global_vf.write_csv('./../data/processed/DWH/recuperadas_global_vf.csv')
    #recuperadas_global_vf_bancomext.write_csv('./../data/processed/DWH/recuperadas_global_vf_bancomext.csv')
    #recuperadas_global_vf_sin_bancomext.write_csv('./../data/processed/DWH/recuperadas_global_vf_sin_bancomext.csv')
    #valida_dwh_dac.write_csv('./../data/processed/DWH/valida_dwh_dac.csv')
    #valida_td.write_csv('./../data/processed/DWH/valida_td.csv')

    return recuperadas_global_vf

In [ ]:
main_querie_recuperaciones()

Producto,DESCRIPCION,ESTATUS,FECHA,FECHA_APERTURA,FECHA_CONSULTA,FECHA_GARANTIA_HONRADA,FECHA_REGISTRO,Fecha Registro Alta,HISTORICO,ID,INTERMEDIARIO_ID,MONEDA_ID,Empresa / Acreditado (Descripción),NUMERO_CREDITO,PORCENTAJE_GARANTIZADO,PROGRAMA_ID,PROGRAMA_ORIGINAL,Razón Social (Intermediario),RFC Empresa / Acreditado,Tipo_Cambio_Cierre,TIPO_CREDITO_ID,TIPO_GARANTIA_ID,TIPO_PERSONA,Gastos Juicio,Moratorios,Interes Generado,Interes,Monto,Monto _Credito_Mn,Excedente,Penalizacion,TPRO_CLAVE,AGRUPAMIENTO_ID,ESQUEMA,SUBESQUEMA,AGRUPAMIENTO,Paridad_Peso,NR_R,CSG,FONDOS_CONTRAGARANTIA,Recup_Rescat,TC,MM_UDIS,CSF,Monto_Mn,Interes_Mn,Moratorios_Mn,Excedente_Mn,Gastos_Juicio_Mn,Sub_Total_Mn,Monto_Total_Mn
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],datetime[μs],str,i64,str,i64,str,str,f32,i32,i32,str,str,f32,i32,i32,str,f32,f32,f32,f32,f32,f64,f32,f32,i32,f64,str,str,str,f64,str,str,str,str,f64,i32,str,f64,f64,f64,f64,f64,f64,f64
"""GARANTIA EMERGENTES""","""CS P.253 - 1063733 DEP 2408201…","""CS""",2010-08-24 00:00:00,2008-02-21 00:00:00,2025-03-31 00:00:00,2010-02-15 00:00:00,2010-08-24 00:00:00,2008-05-15 00:00:00,"""H""",2,"""10040200""",1,"""ANA PATRICIA MARTINEZ VELAZQUE…","""12795429602668697""",100.0,3253,3999,"""BANCO AZTECA""","""MAVA-740317-""",1.0,25,3,"""F""",0.0,null,0.0,-225.639999,-8744.450195,25291.67,0.0,0.0,3253,1335.0,"""PP""","""Tradicional""","""Emergente 2007 Tabasco""",3.96304,"""NR""","""CG""",null,"""Rescatada""",1.0,0,"""CF""",-8744.450195,-225.639999,null,0.0,0.0,-8970.089844,-8970.089844
"""GARANTIA EMERGENTES""","""CS P.253 - 1063733 DEP 2408201…","""CS""",2010-08-24 00:00:00,2008-03-03 00:00:00,2025-03-31 00:00:00,2010-05-10 00:00:00,2010-08-24 00:00:00,2008-04-18 00:00:00,"""H""",2,"""10040200""",1,"""JUDITH VILLEGAS OVANDO""","""12747509602670610""",100.0,3253,3999,"""BANCO AZTECA""","""VIOJ-640719-""",1.0,25,3,"""F""",0.0,null,0.0,-220.789993,-18881.779297,80933.33,0.0,-308.279999,3253,1335.0,"""PP""","""Tradicional""","""Emergente 2007 Tabasco""",3.96604,"""NR""","""CG""",null,"""Rescatada""",1.0,0,"""CF""",-18881.779297,-220.789993,null,0.0,0.0,-19102.568359,-19102.568359
"""GARANTIA EMERGENTES""","""CS P.253 - 1063733 DEP 2408201…","""CS""",2010-08-24 00:00:00,2008-03-04 00:00:00,2025-03-31 00:00:00,2010-03-03 00:00:00,2010-08-24 00:00:00,2008-04-18 00:00:00,"""H""",4,"""10040200""",1,"""FEDERICO GARCIA VALENCIA""","""12722069602671071""",100.0,3253,3999,"""BANCO AZTECA""","""GAVF-740908-""",1.0,25,3,"""F""",0.0,null,0.0,-152.100006,-15713.110352,60700.0,0.0,-407.679993,3253,1335.0,"""PP""","""Tradicional""","""Emergente 2007 Tabasco""",3.9664,"""NR""","""CG""",null,"""Rescatada""",1.0,0,"""CF""",-15713.110352,-152.100006,null,0.0,0.0,-15865.209961,-15865.209961
"""GARANTIA EMERGENTES""","""RP P.253 - 1063733 DEP 1412200…","""D""",2009-12-29 00:00:00,2008-01-07 00:00:00,2025-03-31 00:00:00,2009-10-06 00:00:00,2009-12-29 00:00:00,2008-07-08 00:00:00,"""H""",1,"""10040200""",1,"""MARIO GARCIA GERONIMO""","""12762089602662070""",100.0,3253,3999,"""BANCO AZTECA""","""GAGM-500106-""",1.0,25,3,"""F""",null,null,null,0.0,169.600006,22256.67,0.0,null,3253,1335.0,"""PP""","""Tradicional""","""Emergente 2007 Tabasco""",3.93636,"""NR""","""CG""",null,"""Recuperada""",1.0,0,"""CF""",169.600006,0.0,null,0.0,null,169.600006,null
"""GARANTIA EMERGENTES""","""Recuperacion automatica, carga…","""D""",2010-09-06 00:00:00,2008-03-18 00:00:00,2025-03-31 00:00:00,2010-08-03 00:00:00,2010-09-07 00:00:00,2008-04-18 00:00:00,"""H""",1,"""10040200""",1,"""VICTOR MANUEL FALCON CALCANEO""","""12762089602672517""",100.0,3253,3999,"""BANCO AZTECA""","""FACV-460115-""",1.0,25,3,"""F""",0.0,null,0.01,0.0,3.24,20233.33,0.0,0.0,3253,1335.0,"""PP""","""Tradicional""","""Emergente 2007 Tabasco""",3.97476,"""NR""","""CG""",null,"""Recuperada""",1.0,0,"""CF""",3.24,0.0,null,0.0,0.0,3.24,3.24
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""GARANTI

### Querie Union Flujos

#### Ejecución

In [ ]:
def main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf):
    pagos_agrup = genera_pagos_agrup(pagadas_global_vf)
    pagos_agrup = genera_concatenado(pagos_agrup)

    recuperadas_global_vf = genera_concatenado(recuperadas_global_vf)
    recuperaciones_agrup = genera_recuperaciones_agrup(recuperadas_global_vf)

    # Falta depurar columnas de B
    uf_pagos_recuperaciones = pagos_agrup.join(recuperadas_global_vf, on="Concatenado", how='left')

    # Falta depurar columnas de B
    uf_recuperaciones_pagos = recuperadas_global_vf.join(pagos_agrup, on="Concatenado", how='left')

    t_roberto = pagos_agrup.join(recuperaciones_agrup, on="Concatenado", how='left')

    recupera_con_pagos_flujos = genera_recupera_con_pagos_flujos(uf_recuperaciones_pagos)
    recupera_con_pagos_flujos_bmxt = genera_recupera_con_pagos_flujos_bancomext(recupera_con_pagos_flujos)
    recupera_con_pagos_flujos_sin_bmxt = genera_recupera_con_pagos_flujos_sin_bancomext(recupera_con_pagos_flujos)

    valida_base_pagos_recup = genera_valida_base_pagos_recup(recuperadas_global_vf)
    valida_base_pagos_recup_mn = genera_valida_base_pagos_recup_mn(recuperadas_global_vf)
    
    recupera_con_pagos_flujos.write_csv('./../data/processed/DWH/python/recupera_con_pagos_flujos.csv')
    recupera_con_pagos_flujos_bmxt.write_csv('./../data/processed/DWH/python/recupera_con_pagos_flujos_bmxt.csv')
    recupera_con_pagos_flujos_sin_bmxt.write_csv('./../data/processed/DWH/python/recupera_con_pagos_flujos_sin_bmxt.csv')
    valida_base_pagos_recup.write_csv('./../data/processed/DWH/python/valida_base_pagos_recup.csv')
    valida_base_pagos_recup_mn.write_csv('./../data/processed/DWH/python/valida_base_pagos_recup_mn.csv')

    return 0


### Ejecución Principal

In [ ]:
if __name__ ==  "__main__":
    pagadas_global_vf = main_querie_pagadas()
    recuperadas_global_vf = main_querie_recuperaciones()
    main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)

In [ ]:
end_time = time.time()
end_time - start_time